## PRACTICA OBTENCIÓN DE DATOS

### Web scraper de la página: http://gestiona.madrid.org/azul_internet/html/web/DatosEstacion24Accion.icm?ESTADO_MENU=2_1

#### Libraries

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

#### CODE

In [2]:
#Primero importamos el html de la página web
website = requests.get("http://gestiona.madrid.org/azul_internet/html/web/DatosEstacion24Accion.icm?ESTADO_MENU=2_1")

#Ahora lo guardamos con un objeto "soup"
website_soup = bs(website.content)

#Finalmente imprimimos para ver que tenemos
print(website_soup.prettify())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
 <head>
  <title>
   madrid.org - Comunidad de Madrid
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="es" http-equiv="Content-Language"/>
  <meta content="Web oficial del gobierno autonómico con información sobre economía, educación, servicios sociales,..." name="Description"/>
  <meta content="Comunidad de Madrid" name="Keywords"/>
  <meta content="Comunidad de Madrid" name="Author"/>
  <meta content="oficina.atencion.ciudadano@madrid.org" name="Owner"/>
  <!--link rel="stylesheet" type="text/css" href="../../style/intranet.css"-->
  <link href="../../style/sistemas.css" rel="stylesheet" type="text/css"/>
  <link href="../../style/estilos.css" rel="stylesheet" type="text/css"/>
  <link href="../../style/solapas.css" rel="stylesheet" type="text/css"/>
  <script language="JavaScript" src="../../js/solapas.js" type="text/javascri

In [5]:
#Primero probamos a scrapear una de las tablas de forma sencilla
table = website_soup.find("table")
print(table)

# Esto scrapea solo la primera tabla encontrada

<table border="0" cellpadding="0" cellspacing="0" style="width:980px">
<tr>
<td style="padding-bottom: 10px;" width="89">
<!--      <img style="cursor: pointer;" -->
<!--       		src="../../images/portal/PubliPositivo_15mm_ancho.jpg" alt="Comunidad de Madrid - madrid.org" title="Comunidad de Madrid - madrid.org" width="89" height="64"/> -->
<a href="http://www.comunidad.madrid" target="_blank">
<img alt="Comunidad de Madrid" height="64" src="/webutils/logoCM-izq-89x64.png" style="cursor: pointer;" title="Comunidad de Madrid" width="89"/>
</a>
</td>
<td class="nihil" width="15">
       
    </td>
<td>
<table border="0" cellpadding="0" cellspacing="0" width="100%">
<!--         <tr> -->
<!--           <td colspan="2" class="txt08gr3"> -->
<!--           </td> -->
<!--         </tr> -->
<tr>
<td class="titGeo11azu">
            Red de Calidad del Aire de la Comunidad de Madrid
          </td>
<td id="tdalerta">
</td>
</tr>
<tr>
<td class="puntos" colspan="2">
</td>
</tr>
<tr>
<td colspan=

In [6]:
#Vamos a probar con un find all
tables = website_soup.find_all("table")
print(tables)

#Esto ha creado una lista de todas las tablas

[<table border="0" cellpadding="0" cellspacing="0" style="width:980px">
<tr>
<td style="padding-bottom: 10px;" width="89">
<!--      <img style="cursor: pointer;" -->
<!--       		src="../../images/portal/PubliPositivo_15mm_ancho.jpg" alt="Comunidad de Madrid - madrid.org" title="Comunidad de Madrid - madrid.org" width="89" height="64"/> -->
<a href="http://www.comunidad.madrid" target="_blank">
<img alt="Comunidad de Madrid" height="64" src="/webutils/logoCM-izq-89x64.png" style="cursor: pointer;" title="Comunidad de Madrid" width="89"/>
</a>
</td>
<td class="nihil" width="15">
       
    </td>
<td>
<table border="0" cellpadding="0" cellspacing="0" width="100%">
<!--         <tr> -->
<!--           <td colspan="2" class="txt08gr3"> -->
<!--           </td> -->
<!--         </tr> -->
<tr>
<td class="titGeo11azu">
            Red de Calidad del Aire de la Comunidad de Madrid
          </td>
<td id="tdalerta">
</td>
</tr>
<tr>
<td class="puntos" colspan="2">
</td>
</tr>
<tr>
<td colspan

#### Vamos a extraer las dos tablas que necesitamos. Analizando la lista anterior, determinamos la posición.

In [11]:
#Tabla 1_ es la número tres de la lista y por tanto [2]
table_one = website_soup.select("table")[2]
print(table_one.prettify())

<table align="center" border="0" cellpadding="4" cellspacing="1" width="94%">
 <thead>
  <tr>
   <td align="center" class="txt08gr3" colspan="8" id="fondoVainilla">
    Ultimas 24 Horas
   </td>
  </tr>
  <tr align="center" valign="middle">
   <td>
   </td>
   <td align="center" class="txt08bla" colspan="7" id="fondoGrisMedio">
    <strong>
     Contaminantes
    </strong>
   </td>
  </tr>
  <tr>
   <td>
   </td>
   <td align="center" class="txt07azu" id="fondoGris">
    <a href="javascript:abrirVentana('DatosEstacionGraficoAccion.icm?idAnalizador=7&amp;limSup=25&amp;limInf=0&amp;fechaIni=2021/12/23&amp;fechaFin=2021/12/24&amp;estacion=Getafe&amp;analizador=Temperatura interior cabina', 'wcTIN', 720, 480);" title="Temperatura interior cabina. Pulse con el ratón para ver la gráfica de evolución.">
     <strong>
      TIN
      <br/>
     </strong>
     <small>
      (ºC)
     </small>
    </a>
   </td>
   <td align="center" class="txt07azu" id="fondoGris">
    <a href="javascript:abrirV

In [30]:
#Vamos a coger las columnas de la primera tabla
columns_one = table_one.find("thead").find_all("a")

columns_one_names = [c.text for c in columns_one]

#Y vamos a añadir la primera columna del dataframe
columns_one_names.insert(0, "hours")
print(columns_one_names)

['hours', '\nTIN\n(ºC)', '\nNO\n(µg/m3)', '\nNO2\n(µg/m3)', '\nPM2.5\n(µg/m3)', '\nPM10\n(µg/m3)', '\nNOX\n(µg/m3)', '\nO3\n(µg/m3)']


In [41]:
#Vamos a obtener ahora todas las filas de la tabla 1
rows_one = table_one.find("tbody").find_all("tr")
l = []
for tr in rows_one:
    td = tr.find_all('td')
    row = [str(tr.text).strip() for tr in td]
    l.append(row)

#Y ahora creamos un dataframe con Pandas
df_tabla1 = pd.DataFrame(l, columns = columns_one_names)

#Y finalmente lo guardamos a un .csv
df_tabla1.to_csv("contaminantes.csv")

#Comprobamos el correcto formato
df_tabla1.head()

,hours,\nTIN\n(ºC),\nNO\n(µg/m3),\nNO2\n(µg/m3),\nPM2.5\n(µg/m3),\nPM10\n(µg/m3),\nNOX\n(µg/m3),\nO3\n(µg/m3)
0,23:00,20.6,1,21,11,14,24,21
1,24:00,20.5,1,16,11,13,18,25
2,01:00,20.3,1,15,10,8,18,25
3,02:00,20.3,1,14,15,12,16,24
4,03:00,20.4,1,17,14,18,19,17


In [42]:
#Ahora repetimos todo el proceso junto para la tabla 2
table_two = website_soup.select("table")[3]

columns_two = table_two.find("thead").find_all("a")

columns_two_names = [c.text for c in columns_two]

columns_two_names.insert(0, "hours")

rows_two = table_two.find("tbody").find_all("tr")

l2 = []
for tr in rows_two:
    td = tr.find_all('td')
    row = [str(tr.text).strip() for tr in td]
    l2.append(row)

df_tabla2 = pd.DataFrame(l2, columns = columns_two_names)
df_tabla2.head()

df_tabla2.to_csv("parametrosmeteo.csv")

#Comprobamos el correcto formato
df_tabla2.head()

,hours,\nVV\n(m/s),\nDV\n(Grd),\nTMP\n(ºC),\nHR\n(%),\nPRE\n(mbar),\nRS\n(W/m2),\nLL\n(l/m2)
0,23:00,0.2,141,9.4,94,948,0,0.00
1,24:00,0.5,158,9.4,94,948,0,0.00
2,01:00,0.5,160,9.4,93,947,0,0.10
3,02:00,0.5,162,9.3,93,947,0,0.00
4,03:00,0.2,161,9.2,94,946,0,0.10
